# 启用动态字段

本页说明如何在集合中使用动态栏位以进行弹性资料插入与撷取。

## 概述

在 Milvus 中，动态字段（Dynamic Fields）是指用户可以在不事先定义模式（Schema）的情况下动态添加字段的功能。传统数据库和搜索引擎通常要求在创建模式时明确定义字段的名称和类型，而动态字段则允许在数据集中灵活地添加新的字段，无需修改模式。

动态字段在某些应用场景下非常有用，特别是当数据集的结构不断变化或不确定时。例如，在大规模图像、文本或其他类型的非结构化数据集中，数据项可能具有不同的属性集合。使用动态字段，可以轻松地将这些属性添加到数据集中，而无需修改模式或重新加载数据。

Milvus 提供了动态字段的支持，并通过在插入操作中指定字段名称和值的方式实现。当插入包含动态字段的数据时，Milvus 会自动创建相应的字段并存储数据。这使得用户可以轻松地执行动态模式的数据存储和查询操作。

使用动态字段时，需要注意以下几点：

1. 动态字段的名称是字符串类型。您可以使用任意的字符串作为字段名称。
2. 动态字段的值可以是多种类型，如整数、浮点数、布尔值、字符串等。
3. 动态字段在查询时可以被用作过滤条件，以便只返回符合条件的数据项。
4. 动态字段不支持索引，因此在使用动态字段进行查询时，可能会有性能上的影响。如果需要高性能的查询，建议使用固定的字段和索引。

需要注意的是，动态字段功能在 Milvus 中是作为实验性功能引入的，因此可能存在一些限制和稳定性方面的考虑。在使用动态字段时，建议仔细评估您的应用需求，并在实际部署之前进行充分的测试和验证。

模式设计是Milvus数据处理的关键。在将实体插入到集合之前，请阐明模式设计，并确保之后插入的所有数据实体都与模式匹配。然而，这对集合施加了限制，使它们类似于关系数据库中的表。

动态架构使用户能够将具有新字段的实体插入到集合中，而无需修改现有架构。这意味着用户可以在不知道集合的完整架构的情况下插入数据，并且可以包括尚未定义的字段。

动态模式还提供了数据处理的灵活性，使用户能够在其集合中存储和检索复杂的数据结构。这包括嵌套数据、数组和其他复杂数据类型。

此页面上的代码片段使用新[的MilvusClient](https://milvus.io/api-reference/pymilvus/v2.4.x/About.md)（Python）与Milvus交互。其他语言的新MilvusClient SDK将在未来的更新中发布。


## 启用动态字段

要使用动态模式创建集合，请在定义数据模型时将`enable_dynamic_field`设置为`True`。之后，所有未定义的字段及其在之后插入的数据实体中的值将被视为预定义字段。我们更喜欢使用术语"动态字段"来指代这些键值对。

使用这些动态字段，您可以要求Milvus在搜索/查询结果中输出动态字段，并将其包含在搜索和查询过滤器表达式中，就像它们已经在集合模式中定义一样。


- [dataset](https://www.kaggle.com/datasets/shiyu22chen/cleaned-medium-articles-dataset?resource=download)

In [15]:
import json, os, time
from pymilvus import MilvusClient, DataType

COLLECTION_NAME="medium_articles_2020" # Set your collection name

# 1. Connect to cluster
client = MilvusClient(
    uri="http://localhost:19530"
)

if client.has_collection(COLLECTION_NAME):
    client.drop_collection(COLLECTION_NAME)

# 2. Define collection schema
schema = MilvusClient.create_schema(
    auto_id=True,
    enable_dynamic_field=True
)

schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="title", datatype=DataType.VARCHAR, max_length=512)
schema.add_field(field_name="title_vector", datatype=DataType.FLOAT_VECTOR, dim=768)

# 3. Define index parameters
index_params = MilvusClient.prepare_index_params()

index_params.add_index(
    field_name="title_vector",
    index_type="AUTOINDEX",
    metric_type="L2"
)

# 4. Create a collection
client.create_collection(
    collection_name=COLLECTION_NAME,
    schema=schema,
    index_params=index_params
)

In [20]:
schema

{'auto_id': True, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'title', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 512}}, {'name': 'title_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 768}}], 'enable_dynamic_field': True}

## 插入动态数据

创建集合后，就可以开始将数据（包括动态数据）插入到集合中。

### 准备数据

现在我们需要准备一份适用的数据。

In [16]:
import pandas as pd

In [44]:
df = pd.read_csv('/home/data/New_Medium_Data.csv')

data = df.drop("id", axis=1).to_dict("records")
_ = [item.update({"title_vector": eval(item.get("title_vector"))[:768]}) for item in data]
data[0].keys()

dict_keys(['title', 'title_vector', 'link', 'reading_time', 'publication', 'claps', 'responses'])

### 插入数据

然后，您可以安全地将数据插入到集合中。

In [48]:
# 7. Insert data
res = client.insert(
    collection_name=COLLECTION_NAME,
    data=data,
)

## 使用动态字段搜索

如果您创建的集合启用了动态字段并插入了非架构定义的字段，则可以在搜索或查询的筛选器表达式中使用这些字段，如下所示：


In [52]:
# 8. Search data
res = client.search(
    collection_name=COLLECTION_NAME,
    data=[data[0]['title_vector']],
    filter='claps > 30 and reading_time < 10',
    limit=3,
    output_fields=["title", "reading_time", "claps"],
    search_params={"metric_type": "L2", "params": {}}
)

res

[[{'id': 449793705411903855,
   'distance': 0.36103832721710205,
   'entity': {'title': 'The Hidden Side Effect of the Coronavirus',
    'reading_time': 8,
    'claps': 83}},
  {'id': 449793705411903889,
   'distance': 0.3767402172088623,
   'entity': {'title': 'Why The Coronavirus Mortality Rate is Misleading',
    'reading_time': 9,
    'claps': 2900}},
  {'id': 449793705411901689,
   'distance': 0.4162980318069458,
   'entity': {'title': 'Coronavirus shows what ethical Amazon could look like',
    'reading_time': 4,
    'claps': 51}}]]


## Recaps

值得注意的是，clap和阅读_time在定义模式时并不存在，但这并不妨碍您在过滤器表达式中使用它们，并在插入的数据实体具有这些字段时将它们包含在输出字段中，就像您过去通常所做的那样。

如果动态字段的键包含数字、字母和下划线以外的字符（例如加号、双引号或美元符号），则在布尔表达式中使用它或在输出字段中包含它时，需要将键包含在$meta []中，如以下代码片段所示。

```python
... 
expr='$meta["#key"] in ["a", "b", "c"]', 
output_fields='$meta["#key"]'  
...
```

-----